# Lab02: Frequent itemset mining

- Student ID: 21127229
- Student name: Dương Trường Bình

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example:

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29

30 31 32

33 34 35

36 37 38 39 40 41 42 43 44 45 46

38 39 47 48

38 39 48 49 50 51 52 53 54 55 56 57 58

32 41 59 60 61 62

3 39 48

63 64 65 66 67 68



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [ ]:
from collections import defaultdict

### Read data
First we have to read data from database

In [ ]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.

    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) # a python set is a native way for storing an itemset.
            for item in itemset:
                s[item]+=1     #Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1

    return data, s

### Tree Projection

**I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**TODO:**

In [ ]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)

    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    # TODO (hint: this function will be called in generateSearchSpace method.):
    return set(a).union(set(b))

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}

        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        # TODO: why should we do this, answer it at the markdown below?

        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()

    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}

        search_space = {}
        for item, support in self.s.items():
            search_space[item] = {}

            search_space[item]['itemset'] = [item]
            '''
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.

            search_space[item]['pruned'] = False
            # TODO:
            # After finish implementing the algorithm tell me why should you use this
            # instead of delete item directly from search_space and tree.

            search_space[item]['support'] = support

            tree[item] = {}
            '''
            Why should i store an additional tree (here it called tree)?
            Answer: This really help in next steps.

            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            '''

        return tree, search_space

    def computeItemsetSupport(self, itemset):

        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        support = 0
        for tid, data in self.data.items():
            if itemset.issubset(data):
                support += 1
        return support

    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 0:
            return search_space[itter_node]['support']
        subtree = search_space[itter_node]
        for node in subtree.keys():
            k-=1
            self.get_sub_tree(k,tree,search_space,node)


    def prune(self, k, tree, search_space):

        '''
        In this method we will find out which itemset in current search space is frequent
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        if self.L.get(k) is None: self.L[k] = []
        # TODO
        for node in tree.keys():
            if search_space[node]['support'] < self.minSup:
                search_space[node]['pruned'] = True
            else:
                self.L[k].append(search_space[node]['itemset'])



    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        ''' print search_space.keys() you will understand
        why we need an additional tree, '''
        l = len(items)
        self.prune(k, tree, search_space)
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            if search_space[a]['pruned']: continue

            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).

                # TODO:
                # First create newset using join set
                newset = joinset(search_space[a]['itemset'], search_space[b]['itemset'])

                # Second add newset to search_space
                search_space[a][b] = {
                    'itemset': newset,
                    'pruned': False,
                    'support': self.computeItemsetSupport(newset)
                }
            sub_search_space = search_space[a]
            sub_tree = tree[a]

            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)


    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)

    def miningResults(self):
        return self.L

Ok, let's test on a typical dataset `chess`.

In [ ]:
data, s= readData('chess.txt')

In [ ]:
#
a=TP(data=data,s=s, minSup=3000)
print(a.miningResults())

{1: [[48], [56], [66], [34], [62], [7], [36], [60], [40], [29], [52], [58]], 2: [{48, 52}, {48, 58}, {56, 29}, {56, 52}, {56, 58}, {66, 60}, {66, 29}, {66, 52}, {66, 58}, {40, 34}, {34, 29}, {34, 52}, {34, 58}, {60, 62}, {40, 62}, {29, 62}, {52, 62}, {58, 62}, {60, 7}, {40, 7}, {29, 7}, {52, 7}, {58, 7}, {36, 60}, {40, 36}, {36, 29}, {36, 52}, {58, 36}, {40, 60}, {60, 29}, {60, 52}, {58, 60}, {40, 29}, {40, 52}, {40, 58}, {52, 29}, {58, 29}, {58, 52}], 3: [{48, 58, 52}, {56, 52, 29}, {56, 58, 29}, {56, 58, 52}, {66, 60, 29}, {66, 60, 52}, {66, 60, 58}, {66, 52, 29}, {66, 58, 29}, {66, 52, 58}, {40, 34, 29}, {40, 34, 52}, {40, 34, 58}, {34, 52, 29}, {34, 58, 29}, {34, 52, 58}, {60, 29, 62}, {60, 62, 52}, {58, 60, 62}, {40, 29, 62}, {40, 52, 62}, {40, 58, 62}, {52, 29, 62}, {58, 29, 62}, {58, 52, 62}, {40, 60, 7}, {60, 29, 7}, {60, 52, 7}, {58, 60, 7}, {40, 29, 7}, {40, 52, 7}, {40, 58, 7}, {52, 29, 7}, {58, 29, 7}, {58, 52, 7}, {40, 36, 60}, {36, 29, 60}, {36, 60, 52}, {58, 36, 60}, {40

### Answer questions here:

**why should we do sort**

Trả lời:

Trong hàm `__init__` của class `TP` ta đã dùng hàm `sorted` để sắp xếp các phần tử của dictionary `s` tăng dần theo support của chúng, đảm bảo rằng các itemset có support nhỏ hơn sẽ được xử lý trước. Việc sắp xếp này giúp cải thiện hiệu suất của thuật toán vì chúng ta có thể tỉa bớt các node không thỏa minsup trước mà không sinh ra các nhánh con không cần thiết từ đó giảm bớt thời gian xử lý. Chúng ta đã lưu kết quả sắp xếp vào biến `self.s` để dễ dàng truy cập và sử dụng trong các hàm ở các bước tiếp theo.

**study about python set and its advantages ?**

Trả lời:

`Set` trong Python là một cấu trúc dữ liệu không có thứ tự, không thể thay đổi và không chứa các phần tử trùng lặp. Một số ưu điểm của `set` so với các cấu trúc dữ liệu khác như `list` là:

- Tốc độ truy cập nhanh: Ta có thể truy cập một phần tử thông qua các key mà không cần index như `list`. Thời gian truy cập của `set` là O(1) trong khi của `list` là O(n).

- Các phép toán set: `set` hỗ trợ nhiều phép toán của tập hợp như union, intersection, difference, ... giúp giảm thiểu thời gian xử lý so với việc sử dụng `list`. Trong đó, phép toán `union` giữa hai `set` là rất nhanh chóng và đã được ta dùng trong hàm `joinset` để kết hợp các k-itemset thành k+1-itemset.

- Loại bỏ các phần tử trùng lặp: nếu thêm một phần tử đã tồn tại vào `set`, nó sẽ không thêm vào mà giữ nguyên giá trị cũ (cũng góp phần trong việc kết hợp các itemset trong hàm `joinset`).

- `Set` thích hợp cho việc kiểm tra sự tồn tại của một phần tử trong một tập hợp lớn và ta đã sử dụng `set` trong việc tính toán support của các itemset (hàm `computeItemsetSupport`) với hàm `issubset` để kiểm tra xem itemset có tồn tại trong một transaction hay không một cách nhanh hơn nhiều so với việc sử dụng `list`.

**After finish implementing the algorithm tell me why should you use this? Instead of delete item directly from search_space and tree.**

Trả lời:

Vì điều này giúp ta duy trì cấu trúc của `search_space` và `tree` nguyên vẹn (có thể hữu ích trong việc debug và giúp duy trì tính toàn vẹn dữ liệu). Ngoài ra, việc xóa phần tử trực tiếp từ `search_space` và `tree` có thể gây ra lỗi trong quá trình duyệt và xử lý dữ liệu, đặc biệt là trong các bước tiếp theo của thuật toán. Thay vì xóa trực tiếp, việc đánh dấu chúng là "pruned" giúp tiết kiệm thời gian xử lý và giảm thiểu lỗi.


**Apriori algorithm and Tree Projection, tell me the differences of two algorithms.**

Trả lời:

|-|Apriori algorithm|Tree Projection|
|-|-----------------|---------------|
|Cách tiếp cận| Sử dụng phương pháp tìm kiếm theo cấp độ (level-wise approach) để tạo ra tất cả các itemset ở level k+1 từ các itemset ở level k trước khi kiểm tra support của chúng.| Tiếp cận dựa trên cấu trúc cây, sinh ra tất cả các tập con có thể của level k+1 có chung tiền tố tại node N trước khi kiểm tra support của chúng.|
|Ý tưởng duyệt|Ý tưởng giống thuật toán BFS (Breadth-First Search), duyệt qua từng level k trước khi tiếp tục sang level k+1|Ý tưởng giống thuật toán DFS (Depth-First Search), tại mỗi node X ở level k, ta duyệt sâu vào tất cả các tập con có thể của level k+1 có chung tiền tố với X trước khi xem xét các node khác ở level k|
|Kiểm tra điều kiện join|Cần kiểm tra các item từ 1 đến k-1 của các tập con ở level k trước khi join 2 tập con lại để tạo ra tập con có k+1 item|Do lưu trữ một cây tiền tố, không cần kiểm tra điều kiện như Apriori mà vẫn có thể sinh ra được tất cả các tập con tiếp theo|
|Đặc điểm|Dễ hiểu, dễ cài đặt, và có thể cài đặt song song (parallelize)|Dễ hiểu nhưng cài đặt phức tạp hơn so với Apriori, không thể cài đặt song song nhưng có thể có hiệu suất cao hơn khi xử lý dữ liệu lớn do sử dụng cấu trúc cây tìm kiếm|




# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes).

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.

#### Tìm hiểu dataset churn

Mỗi dòng trong dataset mô tả thông tin của khách hàng, các dịch vụ họ sử dụng và họ ngừng sử dụng dịch vụ của công ty hay không

Ta sẽ dùng thuật toán khai thác tập phổ biến để tìm các đặc trưng thường xuất hiện ở các khách hàng ngừng (không ngừng) sử dụng dịch vụ của công ty

#### Đọc dữ liệu

Vì file `churn.txt` chưa phải là một transaction database nên ta sẽ phải đưa dữ liệu về dạng transaction lúc đọc data.


Hàm đọc dữ liệu sẽ xem mỗi dòng là một transaction, mỗi thuộc tính trong dòng là 1 item nhưng có thêm tiền tố là tên của thuộc tính phía trước
Ví dụ 1 transaction có 2 thuộc tính `Phone`,`Area Code` có giá trị lần lượt là '1234' và '150' sẽ được chuyển đổi thành: ['Phone=1234', 'Area Code= 150']

In [ ]:
# TODO
# YOUR CODE HERE

def readChurn(path):
    data = {}
    s = defaultdict(lambda: 0)
    with open(path, 'rt') as f:
        columns = f.readline().strip().split(',')
        for tid, line in enumerate(f, start=1):
            itemset = line.strip().split(',')
            data[tid] = set(f'{column}={item}' for column, item in zip(columns, itemset))
            for item in data[tid]:
                s[item] += 1
    return data, s

In [ ]:
transactions, s = readChurn('churn.txt')

Sau khi đã đọc dữ liệu về đúng theo dạng transaction, ta sẽ dùng lại thuật toán Tree Projection ở trên để khai thác tập phổ biến. Ta sẽ tạm thời gán minsup=500 và xem kết quả

In [ ]:
# Run the algorithm and print the results
a=TP(data=transactions,s=s, minSup=500)
for k, itemsets in a.miningResults().items():
    print(f'Frequent {k}-itemsets:')
    for itemset in itemsets:
        print(itemset)
    print()

Frequent 1-itemsets:
['Intl Calls=4']
['Intl Calls=3']
['CustServ Calls=0']
['CustServ Calls=2']
['Area Code=408']
['Area Code=510']
['VMail Plan=yes']
['CustServ Calls=1']
['Area Code=415']
['VMail Message=0']
['VMail Plan=no']
['Churn?=False.']
["Int'l Plan=no"]

Frequent 2-itemsets:
{'Intl Calls=4', 'Churn?=False.'}
{"Int'l Plan=no", 'Intl Calls=4'}
{'Intl Calls=3', 'Churn?=False.'}
{"Int'l Plan=no", 'Intl Calls=3'}
{'VMail Message=0', 'CustServ Calls=0'}
{'CustServ Calls=0', 'VMail Plan=no'}
{'Churn?=False.', 'CustServ Calls=0'}
{"Int'l Plan=no", 'CustServ Calls=0'}
{'VMail Message=0', 'CustServ Calls=2'}
{'VMail Plan=no', 'CustServ Calls=2'}
{'Churn?=False.', 'CustServ Calls=2'}
{"Int'l Plan=no", 'CustServ Calls=2'}
{'VMail Message=0', 'Area Code=408'}
{'VMail Plan=no', 'Area Code=408'}
{'Churn?=False.', 'Area Code=408'}
{"Int'l Plan=no", 'Area Code=408'}
{'Area Code=510', 'VMail Message=0'}
{'Area Code=510', 'VMail Plan=no'}
{'Area Code=510', 'Churn?=False.'}
{"Int'l Plan=no", 'A

Vậy với minsup=500 ta đã khai thác được các tập phổ biến trên

Ta có thể giải thích kết quả của 2 tập phổ biến ở 5-itemsets:

- Các khách hàng mà không dùng dịch vụ quốc tế, không có lời nhắn thoại nào, không dùng voice mail,  có 1 cuộc gọi đến dịch vụ khách hàng thì thường sẽ ngừng dùng dịch vụ của công ty

- Các khách hàng mà không dùng dịch vụ quốc tế, không có lời nhắn thoại nào, không dùng voice mail,  có mã vùng là 415 thì thường sẽ ngừng dùng dịch vụ của công ty

Ngoài ra ta có thể tùy chỉnh các ngưỡng minsup khác để xem xét các quả phù hợp khác nhau cho các minsup đó.